In [3]:


DEFAULT_TIMEOUT = 1  # seconds

# class TimeoutHTTPAdapter(HTTPAdapter):
#     def __init__(self, *args, **kwargs):
#         self.timeout = DEFAULT_TIMEOUT
#         if "timeout" in kwargs:
#             self.timeout = kwargs["timeout"]
#             del kwargs["timeout"]
#         super().__init__(*args, **kwargs)
#
#     def send(self, request, **kwargs):
#         timeout = kwargs.get("timeout")
#         if timeout is None:
#             kwargs["timeout"] = self.timeout
#         return super().send(request, **kwargs)

In [4]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import requests

http = requests.Session()

retries = Retry(total=3, backoff_factor=0.3, status_forcelist=[429, 500, 502, 503, 504])
http.mount("https://", HTTPAdapter(max_retries=retries))
http.mount("http://", HTTPAdapter(max_retries=retries))

try:
    response = http.get("http://httpbin.org/status/500", timeout=1)
except requests.exceptions.RequestException as e:
    print('Received status 500', e)

try:
    response = http.get("http://httpbin.org/status/200", timeout=1)
    print('Received status 500', response)
except requests.exceptions.RequestException as e:
    print(e)

try:
    response = http.get("http://httpbin.org/delay/10", timeout=1)
except requests.exceptions.RequestException as e:
    print('With 1 seconds timeout', e)

try:
    response = http.get("http://httpbin.org/delay/10", timeout=10)
    print('With 20 seconds timeout', response)
except requests.exceptions.RequestException as e:
    print(e)

Received status 500 HTTPConnectionPool(host='httpbin.org', port=80): Max retries exceeded with url: /status/500 (Caused by ResponseError('too many 500 error responses'))
Received status 500 <Response [200]>
With 1 seconds timeout HTTPConnectionPool(host='httpbin.org', port=80): Max retries exceeded with url: /delay/10 (Caused by ReadTimeoutError("HTTPConnectionPool(host='httpbin.org', port=80): Read timed out. (read timeout=1)"))
HTTPConnectionPool(host='httpbin.org', port=80): Max retries exceeded with url: /delay/10 (Caused by ReadTimeoutError("HTTPConnectionPool(host='httpbin.org', port=80): Read timed out. (read timeout=10)"))


In [1]:


class HTTPWrapper(object):

    def __init__(
            self,
            api_name,
            base_url,
            total=3,
            backoff_factor=10,
            status_forcelist=(429, 500, 502, 503, 504),
            method_whitelist=(
                    'HEAD',
                    'GET',
                    'PUT',
                    'POST',
                    'DELETE',
                    'OPTIONS',
                    'TRACE',
            ),
    ):
        self.api_name = api_name
        self.base_url = base_url
        self.retries = Retry(
            total=total,
            backoff_factor=backoff_factor,
            status_forcelist=status_forcelist,
            method_whitelist=method_whitelist,
        )

    def request(self, method, end_point, url=None, retries=None, **kwargs):
        url = url or self.base_url + end_point
        retries = retries or self.retries

        http = requests.Session()
        http.mount('https://', HTTPAdapter(max_retries=retries))
        http.mount('http://', HTTPAdapter(max_retries=retries))
        request = getattr(http, method.lower())

        try:
            res = request(url=url, **kwargs)
        except requests.exceptions.RequestException as e:
            print(f'Request exception: {e}')

        return res


api = HTTPWrapper(
            api_name='Acurity',
            base_url="http://httpbin.org/",
            total=5,
            backoff_factor=10,
            status_forcelist=(429, 500, 502, 503, 504),
        )

res = api.request(
            method='GET',
            end_point='/delay/10',
        )

print(res)

NameError: name 'Retry' is not defined